## 载入依赖包

In [4]:
import json
import pandas as pd
from ollama import chat
from openai import OpenAI
from tqdm.notebook import tqdm
from sqlalchemy import create_engine, Column, Text, String, Integer, MetaData, Table, select
from sqlalchemy.orm import sessionmaker

## 读取数据集

In [5]:
# 读取JSON文件并创建DataFrame
df = pd.read_json('CMedExamQA.json')
print(df.shape)
df.head(3)

(10694, 11)


,question,opa,opb,opc,opd,ope,answer,year,question_number,source,id
0,小剂量地塞米松抑制试验适用于（ ）。,醛固酮增多症定性,肾上腺皮质增多症定性,肾上腺皮质功能减退症定位,肾上腺皮质功能减退症定性,肾上腺皮质增多症定位,opb,2017.0,1.0,pubmed-38355517;github-llm-chinese-nmle,413f38dc-3df2-5955-8858-16e460462f44
1,下列成分可见于肾小管性蛋白尿的是（ ）。,IgM,补体,IgG,β2微球蛋白,本周蛋白,opd,2017.0,2.0,pubmed-38355517;github-llm-chinese-nmle,45ba857a-8989-5527-89cc-d4320f8b3226
2,原发性肾小球疾病的临床分类不包括（ ）。,无症状性血尿或（和）蛋白尿,慢性肾小球肾炎,肾病综合征,急进性肾小球肾炎,肾盂肾炎,ope,2017.0,3.0,pubmed-38355517;github-llm-chinese-nmle,a0a85ae8-48a3-5371-8e71-a60c600c0269


## 调用大模型，获得response，并存储在SQLite数据库里

In [3]:
gpt_api_key = "sk-****"

In [4]:
# Please install OpenAI SDK first: `pip3 install openai`
def call_openai(model, content):
    client = OpenAI(api_key=gpt_api_key)
    completion = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": content}
      ]
    )
    response_text = completion.choices[0].message.content
    return response_text

### 调用一个样例

In [8]:
model="gpt-4o"

for i, row in df.iterrows():
    qid = row["id"]
    question = row["question"]
    opa = row["opa"]
    opb = row["opb"]
    opc = row["opc"]
    opd = row["opd"]
    ope = row["ope"]
    if qid == "413f38dc-3df2-5955-8858-16e460462f44":
        prompt = f"""请根据以下医学单选题，从五个选项中选择一个最合适的答案，并将答案以 JSON 格式输出，格式如下：

        {{"answer": "X"}}
        
        其中 X 是 A、B、C、D 或 E 中的一个选项。请不要输出任何解释或额外信息。
        
        题目：
        {question}
        A. {opa}
        B. {opb}
        C. {opc}
        D. {opd}
        E. {ope}
        """

        response_text = call_openai(model=model, content=prompt)
        print(response_text)
        break

{"answer": "B"}


In [9]:
# # 调用gpt4o
# for i, row in df.iterrows():
#     qid = row["id"]
#     whole_question = row["whole_question"]
#     if qid == 21962:
#         response_text = call_openai(prompt+whole_question)
#         print(response_text)
#         break

### 批量调用

In [10]:
def batch_call_openai(model):
    # 1. 设置数据库路径（使用 SQLite 示例，也可更换为 MySQL/PostgreSQL）
    DATABASE_URL = "sqlite:///gpt_response.db"
    
    # 2. 初始化数据库连接
    engine = create_engine(DATABASE_URL)
    metadata = MetaData()
    
    # 3. 定义表结构（如果不存在会自动创建）
    response_table = Table(
        model.replace(":","-"), metadata,
        Column('id', String, primary_key=True),
        Column('response', Text),
    )
    metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # 4. 处理每一行数据
    # 读取 JSON 文件为 DataFrame
    df = pd.read_json("CMedExamQA.json")
    for i, row in tqdm(df.iterrows(), total=len(df), desc='Processing'):
        qid = row["id"]
        question = row["question"]
        opa = row["opa"]
        opb = row["opb"]
        opc = row["opc"]
        opd = row["opd"]
        ope = row["ope"]

        exists = session.query(response_table).filter_by(id=qid).first()
        if exists:
            # if i % 1000 == 0: print(f"跳过已存在记录: {qid}")
            continue
            
        # 调用ollama
        # prompt = f"{question}\nA. {opa}\nB. {opb}\nC. {opc}\nD. {opd}\nE. {ope}"
        prompt = f"""请根据以下医学单选题，从五个选项中选择一个最合适的答案，并将答案以 JSON 格式输出，格式如下：

        {{"answer": "X"}}
        
        其中 X 是 A、B、C、D 或 E 中的一个选项。请不要输出任何解释或额外信息。
        
        题目：
        {question}
        A. {opa}
        B. {opb}
        C. {opc}
        D. {opd}
        E. {ope}
        """

        response_text = call_openai(model=model, content=prompt)
        session.execute(response_table.insert().values(id=qid, response=response_text))

        # 每100条提交一次
        if i % 100 == 0:
            session.commit()
            # print(f"已提交 {i} 条记录")
            
    session.commit()  # ✅ 别忘了 commit
    session.close()

In [11]:
batch_call_openai(model="gpt-4o")

Processing:   0%|          | 0/10694 [00:00<?, ?it/s]

---

In [3]:
from sqlalchemy import create_engine, inspect, text

# 连接数据库（以 SQLite 为例）
DATABASE_URL = "sqlite:///gpt_response.db"
engine = create_engine(DATABASE_URL)

# 创建 Inspector 用于反射
inspector = inspect(engine)

# 获取所有表名
table_names = inspector.get_table_names()
print("📋 数据库中的表：")
print(table_names)

# 获取每个表的行数
with engine.connect() as conn:
    for table in table_names:
        result = conn.execute(text(f"SELECT COUNT(*) FROM '{table}'"))
        count = result.scalar()
        print(f"✅ 表 `{table}` 中共有 {count} 条记录")

for table in table_names:
    print(f"\n🧱 表 `{table}` 的字段信息：")
    columns = inspector.get_columns(table)
    for col in columns:
        print(f"- {col['name']} ({col['type']})")
engine.dispose()

📋 数据库中的表：
['gpt-4o']
✅ 表 `gpt-4o` 中共有 10694 条记录

🧱 表 `gpt-4o` 的字段信息：
- id (VARCHAR)
- response (TEXT)


In [1]:
from sqlalchemy import create_engine, MetaData, Table, select

# 连接数据库
engine = create_engine("sqlite:///gpt_response.db")
metadata = MetaData()
metadata.reflect(bind=engine)

# 反射获取表
response_table = metadata.tables["gpt-4o"]

# 创建连接
with engine.connect() as conn:
    # 1）打印出一行数据
    result = conn.execute(select(response_table).limit(1)).fetchone()
    print("📌 一行数据：", result)

    # 2）根据特定 id 提取 response
    # target_id = 21962  # <-- 替换为你的实际 ID
    # target_id = 22616  # <-- 替换为你的实际 ID
    # target_id = "413f38dc-3df2-5955-8858-16e460462f44"
    # target_id = "45ba857a-8989-5527-89cc-d4320f8b3226"
    target_id = "a0a85ae8-48a3-5371-8e71-a60c600c0269"
    # target_id = 23899  # <-- 替换为你的实际 ID
    stmt = select(response_table.c.response).where(response_table.c.id == target_id)
    response = conn.execute(stmt).scalar()

    if response:
        print(f"📥 ID 为 {target_id} 的 response:")
        print(response)
    else:
        print(f"❌ 未找到 ID 为 {target_id} 的记录")

📌 一行数据： ('413f38dc-3df2-5955-8858-16e460462f44', '{"answer": "B"}')
📥 ID 为 a0a85ae8-48a3-5371-8e71-a60c600c0269 的 response:
{"answer": "E"}


---

## 提取数据库中的response，解析出答案，并保存为txt

In [7]:
import json
import re

def extract_answer_from_response(response_str):
    """
    从模型响应中提取 {"answer": "X"} 格式的答案。
    如果无法提取有效答案，则返回 None。
    """
    try:
        # 使用正则表达式提取 JSON 结构
        json_match = re.search(r'\{.*?\}', response_str, re.DOTALL)
        if json_match:
            json_str = json_match.group()
            # 尝试解析为 JSON
            parsed = json.loads(json_str)
            if isinstance(parsed, dict) and 'answer' in parsed:
                answer = parsed['answer'].strip().upper()
                if answer in ['A', 'B', 'C', 'D', 'E']:
                    return answer
        return "NA"
    except Exception as e:
        print(f"Error parsing response: {e}")
        return "NA"

# 示例响应测试
example_responses = [
    '{"answer": "A"}',
    'Here is the answer:\n{"answer": "C"}',
    '答：{"answer": "D"}。请参考资料。',
    '解释略。\n{"answer": "B"}\n谢谢！',
    '错误格式',
]

for i, res in enumerate(example_responses, 1):
    parsed = extract_answer_from_response(res)
    print(f"Example {i}: {parsed}")

Example 1: A
Example 2: C
Example 3: D
Example 4: B
Example 5: NA


In [9]:
from sqlalchemy import create_engine, MetaData, Table, select

# 连接数据库
engine = create_engine("sqlite:///gpt_response.db")
metadata = MetaData()
metadata.reflect(bind=engine)

# 反射获取表
model_name = "gpt-4o"


llm_table = metadata.tables[model_name]

id_response = []
# 创建连接
i = 0
file_name = "gpt-4o"

with engine.connect() as conn:
    with open(f"{file_name}.txt", 'w') as f:
        # 根据特定 id 提取 response
        for target_id in df["id"]:
            stmt = select(llm_table.c.response).where(llm_table.c.id == target_id)
            response = conn.execute(stmt).scalar()
            
            id_response.append({'id': target_id, 'response': response})
            
            answer = extract_answer_from_response(response)
            if not answer:
                print(target_id)
                i += 1
            if response:
                f.write(target_id + "\t" + answer + "\n")
            else:
                print(f"❌ 未找到 ID 为 {target_id} 的记录")

with open(f"{file_name}.json", 'w') as json_file:
    json.dump(id_response, json_file, indent=4)
print(i)

0
